In [63]:
# Source: Alexandru Tifrea and Fanny Yang, 2021.

# Python Notebook Commands
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from copy import deepcopy
import numpy as np
import time

import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

import ipywidgets
from ipywidgets import interact, interactive, interact_manual

import sklearn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV

from utils import generate_data, generate_additional_data, compute_population_risk, compute_empirical_risk, repeat_experiment

# Change these values if the images don't fit for your screen.
figure_width=1200
figure_height=500

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bias-variance trade-off for ridge regression

We can estimate the bias and the variance of an estimator by sampling different training sets, and using a hold-out validation set to compute its empirical error.

The stacked area plot below illustrates the decomposition of the risk into three terms: the squared bias, the variance, and irreducible noise.

In [62]:
validation_size = 1000
num_trials = 5
snr = 1

all_noise_sigmas = [0, 0.1, 0.5, 1]

def plot_bias_variance_for_ridge(n, d, noise_sigma):
  ridge_coefficients = np.arange(0, 100, 1)
  risks, squared_biases, variances = [], [], []
  n, d = int(n), int(d)
  
  # Sample the validation set and one traing set for each of the trials.
  X_validation, y_validation, beta_star, Sigma = generate_data(n=validation_size, d=d, snr=snr, noise_sigma=noise_sigma)
  all_X, all_y = generate_additional_data(num_samples=n*num_trials, d=d, Sigma=Sigma, beta_star=beta_star, noise_sigma=noise_sigma)
  for ridge_coef in ridge_coefficients:
    validation_predictions, validation_bayes_predictions = [], []
    
    # Train num_trials estimators and use the validation set to estimate the bias and variance.
    for i in range(num_trials):
      start, end = i * n, (i + 1) * n
      ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False).fit(all_X[start:end], all_y[start:end])
      validation_predictions.append((X_validation @ ridge_reg.coef_.T).reshape(-1, 1))
      validation_bayes_predictions.append((X_validation @ beta_star).reshape(-1, 1))
      
    validation_predictions, validation_bayes_predictions = np.array(validation_predictions), np.array(validation_bayes_predictions)
    squared_biases.append(
      np.power(validation_predictions.mean(axis=0) - validation_bayes_predictions, 2).mean()
    )
    variances.append(
      np.power(validation_predictions - validation_predictions.mean(axis=0), 2).mean()
    )
    risks.append(
      np.power(validation_predictions - y_validation, 2).mean()
    )

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=ridge_coefficients, y=np.ones_like(alphas) * noise_sigma**2, name="Irreducible noise", 
                           marker_color="gray", stackgroup='one'))
  fig.add_trace(go.Scatter(x=ridge_coefficients, y=variances, name="Variance", stackgroup='one'))
  fig.add_trace(go.Scatter(x=ridge_coefficients, y=squared_biases, name="Bias<sup>2</sup>", stackgroup='one'))
  fig.add_trace(go.Scatter(x=ridge_coefficients, y=risks, name="Risk", stackgroup='one'))

  if noise_sigma == 0.5:
    yaxis_range = [0, 1]
  elif noise_sigma == 1:
    yaxis_range = [0, 3]
  else:
    yaxis_range = [0, 0.1]
  
  fig.update_layout(
    height=figure_height,
    width=figure_width,
    yaxis_range=yaxis_range,
    yaxis_title="Risk / Bias / Variance",
    xaxis_title="Ridge coefficient",
    hovermode='x'
  )
  fig.show()
  
interact(plot_bias_variance_for_ridge,
         n=ipywidgets.FloatSlider(value=200,
                                  min=100,
                                  max=500,
                                  step=10,
                                  readout_format='d',
                                  description='Number of samples:',
                                  style={'description_width': 'initial'},
                                  continuous_update=False),
         d=ipywidgets.FloatSlider(value=100,
                                  min=10,
                                  max=100,
                                  step=10,
                                  readout_format='d',
                                  description='Data dimension:',
                                  style={'description_width': 'initial'},
                                  continuous_update=False),
         noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                         value=0.5,
                                         description='Noise level:',
                                         disabled=False,
                                         style={'description_width': 'initial'},
                                         continuous_update=True),);

interactive(children=(FloatSlider(value=200.0, continuous_update=False, description='Number of samples:', max=…